## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-15-18-51-04 +0000,nyt,F.B.I. Burns Methamphetamine Near Animal Shelt...,https://www.nytimes.com/2025/09/15/us/montana-...
1,2025-09-15-18-48-55 +0000,nypost,Idaho small business receives outpouring of su...,https://nypost.com/2025/09/15/us-news/idaho-sm...
2,2025-09-15-18-45-24 +0000,cbc,Government lays out next steps in probe of unp...,https://www.cbc.ca/news/canada/government-labo...
3,2025-09-15-18-42-00 +0000,wsj,Opinion | Russia’s Alarming Drone Incursion In...,https://www.wsj.com/opinion/russias-alarming-d...
4,2025-09-15-18-41-00 +0000,wsj,Opinion | Charlie Kirk’s Assassination Unleash...,https://www.wsj.com/opinion/charlie-kirks-assa...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2290/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
103,trump,37
16,kirk,25
15,charlie,21
139,china,13
178,new,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
231,2025-09-14-23-51-33 +0000,nypost,Trump blasts ‘the left’ and announces ‘major i...,https://nypost.com/2025/09/14/us-news/trump-bl...,99
119,2025-09-15-14-14-04 +0000,nypost,Charlie Kirk shooting is ‘the death of free sp...,https://nypost.com/2025/09/15/media/charlie-ki...,77
61,2025-09-15-16-57-00 +0000,nypost,"On ‘Charlie Kirk Show’ hosted by JD Vance, Ste...",https://nypost.com/2025/09/15/us-news/jd-vance...,76
18,2025-09-15-18-19-31 +0000,nyt,U.S.-China Deal to Avert TikTok Ban May be Clo...,https://www.nytimes.com/2025/09/15/technology/...,76
14,2025-09-15-18-27-22 +0000,wapo,‘It was me’: Suspect in Charlie Kirk shooting ...,https://www.washingtonpost.com/investigations/...,76


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
231,99,2025-09-14-23-51-33 +0000,nypost,Trump blasts ‘the left’ and announces ‘major i...,https://nypost.com/2025/09/14/us-news/trump-bl...
91,49,2025-09-15-15-15-00 +0000,wsj,Canadian factories notched the biggest rise in...,https://www.wsj.com/articles/canada-factory-an...
123,40,2025-09-15-14-03-11 +0000,nypost,"Bessent says US, China have reached ‘framework...",https://nypost.com/2025/09/15/business/us-chin...
95,39,2025-09-15-15-04-49 +0000,nypost,Top GOP foreign-policy Rep. Michael McCaul won...,https://nypost.com/2025/09/15/us-news/texas-re...
115,37,2025-09-15-14-21-52 +0000,bbc,Democratic New York governor endorses Zohran M...,https://www.bbc.com/news/articles/cgl1e4501pgo...
233,36,2025-09-14-23-38-49 +0000,wapo,Kirk shooting suspect had ‘leftist ideology’ b...,https://www.washingtonpost.com/nation/2025/09/...
105,32,2025-09-15-14-39-54 +0000,nypost,Tesla share surge as Elon Musk buys $1B in sto...,https://nypost.com/2025/09/15/business/tesla-s...
29,31,2025-09-15-17-50-03 +0000,wapo,"Rubio downplays U.S. tensions with Israel, pla...",https://www.washingtonpost.com/national-securi...
38,29,2025-09-15-17-34-14 +0000,nypost,Washington Post columnist Karen Attiah claims ...,https://nypost.com/2025/09/15/media/washington...
111,29,2025-09-15-14-27-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
